# GETTIN
## Planejamento de Equipamentos Escolares
## Método: Capacitaded Facility Location
### Versão: 0.9
#### Fillipe O Feitosa <fillipefeitosa@ua.pt>

---


#### Math Programming


\begin{split}& \text{minimizar} \quad    & \sum_{j=1}^m f_j x_j +\sum_{i=1}^n \sum_{j=1}^m c_{ij} y_{ij} &     \\
& \text{sujeito a:} \quad & \sum_{j=1}^m y_{ij} =d_i  &  \mbox{ for }  i=1,\cdots,n \\
&    & \sum_{i=1}^n y_{ij} \leq M_j x_j & \mbox{ para } j=1,\cdots,m  \\
&    & y_{ij} \leq d_i x_j              & \mbox{ para } i=1,\cdots,n; j=1,\cdots,m \\
&    & y_{ij} \geq 0                    & \mbox{ para } i=1,\cdots,n; j=1,\cdots,m \\
&    & x_j \in \{ 0,1 \}                & \mbox{ para } j=1,\cdots,m\end{split}

#### List of Imports

In [1]:
# Import Libraries
from gurobi import *
import math
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import geopy.distance
from collections import defaultdict


#### Problem Data

In [2]:
# ------------------------- Problem data ------------------------- #

# Dict for SubSections With Demand
rawData = pd.read_excel('./subseccao_vagos/Pop_escolar_Vagos.xlsx')
listOfSubsections_2010 = {}
for e in range(len(rawData)-1):
    listOfSubsections_2010[int(rawData.SUBSECCAO[e])] = rawData.P_esc_1CEB_2010[e]+ rawData.P_esc_2CEB_2010[e]+rawData.P_esc_3CEB_2010[e]+rawData.P_esc_secund_2010[e]+rawData.Pre_escolar_2010[e]

# Dict of Lists for Schools with Capacity And Cost by Size
tupleOfCapacity = (80, 120, 200)

schoolsCapacityCost = defaultdict(list)

for bgri in list(listOfSubsections_2010.keys()):
    for capacity in tupleOfCapacity:
        cost = capacity*6200
        tempList = (capacity, cost)
        schoolsCapacityCost[bgri].append(tempList)

# Retrieve Coordinates and Vinculate to Identifier (BGRI)
vagos = gpd.read_file('./data_gettin/vagos.geojson');
centroids = vagos.centroid

iteratorHandler = centroids.size
centroidVector = []
for centroid in centroids:
    obj = [centroid.xy[0][0], centroid.xy[1][0]]
    centroidVector.append(obj)
    
coordinates_X = {}
coordinates_Y = {}
for e in range (len(vagos)):
    coordinates_X[int(vagos.BGRI11[e])] = centroids[e].xy[0][0]
    coordinates_Y[int(vagos.BGRI11[e])] = centroids[e].xy[1][0]
    
coordinates_X[list(listOfSubsections_2010.keys())[0]]

-8.69434885302111

#### Modeling and creation of decision Variables

In [3]:
   
# -------- Distance Matrix Logic ------- #
transportationCosts = {}

for bgri_1 in list(listOfSubsections_2010.keys()):
    coords_1 = (coordinates_X[bgri_1], coordinates_Y[bgri_1])
    for bgri_2 in list(listOfSubsections_2010.keys()):
        coords_2 = (coordinates_X[bgri_2], coordinates_Y[bgri_2])
        # 0.29 cents per KM * 180 days
        cost = geopy.distance.vincenty(coords_1, coords_2).km*0.29*180
        transportationCosts[bgri_1, bgri_2] = cost
        
transportationCosts

{(1180100101, 1180100101): 0.0,
 (1180100101, 1180100102): 10.77046440282093,
 (1180100101, 1180100103): 15.141187051660344,
 (1180100101, 1180100104): 37.92750804142575,
 (1180100101, 1180100105): 24.866757739232916,
 (1180100101, 1180100106): 51.85575868675049,
 (1180100101, 1180100107): 64.53433609446037,
 (1180100101, 1180100108): 61.64927490516296,
 (1180100101, 1180100109): 84.93003200156626,
 (1180100101, 1180100110): 66.99862842601652,
 (1180100101, 1180100111): 75.04170950649684,
 (1180100101, 1180100112): 80.21250268108949,
 (1180100101, 1180100113): 76.05413621376627,
 (1180100101, 1180100114): 85.84933267568505,
 (1180100101, 1180100115): 114.53012577693266,
 (1180100101, 1180100116): 107.31307723591222,
 (1180100101, 1180100117): 37.83327648205344,
 (1180100101, 1180100118): 65.99655851479788,
 (1180100101, 1180100119): 54.19468633049367,
 (1180100101, 1180100120): 81.72344300047425,
 (1180100101, 1180100121): 54.83832021010197,
 (1180100101, 1180100122): 69.37619596649992

In [4]:
# for school in list(schoolsCapacityCost.keys()):
#     for schoolSize in range(len(schoolsCapacityCost[school])):
#         print(schoolsCapacityCost[school][schoolSize][0])
#         print(schoolSize)

In [5]:

# -------- Decision Variables ---------- #

# numSchools = len(escola)
# numSubSections = len(subSecao)

# Creting Guroby Model
m = Model()

# Decision Variables
x = {}
y = {}
# d = {} # Distance Matrix
# @alpha: 0.29 de custo por Km  por (365 dias * 5 anos) 
# alpha = 529.25

# creating binary variable for every school
for j in list(schoolsCapacityCost.keys()):
    for schoolSize in range(len(schoolsCapacityCost[j])):
        x[(j,schoolsCapacityCost[j][schoolSize][0])] = m.addVar(vtype=GRB.BINARY, name="escola(%d,%d)" % (j, schoolsCapacityCost[j][schoolSize][0]))

# creating continuous variable for subsections to check suply fractions
for i in list(listOfSubsections_2010.keys()):
    for j in list(schoolsCapacityCost.keys()):
        # Fractions of Subsection Suply
        y[(i,j)] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Fração da Sub[%d], escola[%d]" % (i,j))

m.update()

In [6]:
# schoolsCapacityCost[1180100101][1]
# listOfSubsections_2010[1180100101]
# for school in list(schoolsCapacityCost.keys()):
#     for schoolSize in range(len(schoolsCapacityCost[school])):
#         sizeOfSchool = schoolsCapacityCost[school][schoolSize][0]
#         print(x[school, sizeOfSchool])


## Adding Constraints

In [7]:
# Constraint for Every Student on School
# for i in range(numSubSections):
#     m.addConstr(quicksum(y[(i,j)] for j in range(numSchools)) == 1)

# Constraint to repect demands on Every Subsection - OK
for i in list(listOfSubsections_2010.keys()):
    m.addConstr(quicksum(y[(i,j)] for j in list(schoolsCapacityCost.keys())) == listOfSubsections_2010[i])
    
# Constraint to restrain school capacity - OK
for school in list(schoolsCapacityCost.keys()):
    for schoolSize in range(len(schoolsCapacityCost[school])):
        numericalSizeOfSchool = schoolsCapacityCost[school][schoolSize][0]
        m.addConstr(quicksum(y[(i,school)] for i in list(listOfSubsections_2010.keys())) <= numericalSizeOfSchool * x[school, numericalSizeOfSchool])
    
# Add restriction to fraction Size - OK
for (i,j) in y:
    for schoolSize in range(len(schoolsCapacityCost[j])):
        numericalSizeOfSchool = schoolsCapacityCost[j][schoolSize][0]
        m.addConstr(y[i,j] <= listOfSubsections_2010[i]*x[j, numericalSizeOfSchool], "Strong(%s,%s)"%(i,j))


#### Objetive


In [ ]:
# Setting objective

m.setObjective(
    quicksum(schoolsCapacityCost[school][schoolSize][1]*x[school, schoolsCapacityCost[school][schoolSize][0]] for school in list(schoolsCapacityCost.keys()) for schoolSize in range(len(schoolsCapacityCost[school]))) + 
    quicksum(transportationCosts[i,j]*y[i,j] for i in list(listOfSubsections_2010.keys()) for j in list(schoolsCapacityCost.keys())), GRB.MINIMIZE )


m.optimize()

Optimize a model with 1223684 rows, 408958 columns and 3974740 nonzeros
Variable types: 407044 continuous, 1914 integer (1914 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+02]
  Objective range  [2e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 5e+01]
Found heuristic solution: objective 9.737527e+08
Presolve removed 97665 rows and 32538 columns (presolve time = 5s) ...
Presolve removed 97665 rows and 32538 columns
Presolve time: 9.87s
Presolved: 1126019 rows, 376420 columns, 3746974 nonzeros
Variable types: 374506 continuous, 1914 integer (1914 binary)

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolved: 1126019 rows, 376420 columns, 3746974 nonzeros

Root barrier log...

Elapsed ordering time = 5s
Elapsed ordering time = 10s
Ordering time: 10.72s

Barrier statistics:
 Dense cols : 1914
 AA' NZ     : 7.868e+06
 Factor NZ  : 1.986e+07 (roughly 800 MBytes of memory)
 Factor Ops 

  95   1.25150240e+08  1.25150151e+08  7.11e-15 6.53e-07  4.73e-05   931s
  96   1.25150237e+08  1.25150153e+08  7.99e-15 6.25e-07  4.50e-05   939s
  97   1.25150230e+08  1.25150158e+08  7.99e-15 5.35e-07  3.85e-05   946s
  98   1.25150229e+08  1.25150159e+08  8.44e-15 5.27e-07  3.73e-05   954s
  99   1.25150227e+08  1.25150160e+08  1.51e-14 5.06e-07  3.54e-05   963s
 100   1.25150224e+08  1.25150162e+08  1.07e-14 4.74e-07  3.31e-05   975s
 101   1.25150219e+08  1.25150169e+08  8.44e-15 3.76e-06  2.67e-05   990s
 102   1.25150211e+08  1.25150171e+08  7.99e-15 3.53e-06  2.17e-05  1002s
 103   1.25150208e+08  1.25150174e+08  9.77e-15 2.91e-06  1.79e-05  1010s
 104   1.25150203e+08  1.25150178e+08  9.77e-15 2.28e-06  1.33e-05  1017s
 105   1.25150201e+08  1.25150180e+08  8.44e-15 1.93e-06  1.13e-05  1024s
 106   1.25150200e+08  1.25150182e+08  3.82e-14 1.66e-06  9.87e-06  1034s
 107   1.25150197e+08  1.25150184e+08  2.13e-14 1.28e-06  7.28e-06  1046s
 108   1.25150195e+08  1.25150187e+08 

     0     0 1.2515e+08    0  381 1.2658e+08 1.2515e+08  1.13%     - 2145s
     0     0 1.2515e+08    0  387 1.2658e+08 1.2515e+08  1.13%     - 2167s
     0     0 1.2515e+08    0  387 1.2658e+08 1.2515e+08  1.13%     - 3360s
     0     2 1.2515e+08    0  387 1.2658e+08 1.2515e+08  1.13%     - 4307s
     1     4 1.2515e+08    1  384 1.2658e+08 1.2515e+08  1.13%   821 4434s
     3     8 1.2515e+08    2  384 1.2658e+08 1.2515e+08  1.13%   597 4605s
     7    12 1.2515e+08    3  384 1.2658e+08 1.2515e+08  1.13%   465 4679s
    11    16 1.2515e+08    3  366 1.2658e+08 1.2515e+08  1.13%   580 4763s
    15    18 1.2515e+08    4  375 1.2658e+08 1.2515e+08  1.13%   568 4781s
    19    21 1.2515e+08    5  357 1.2658e+08 1.2515e+08  1.13%   463 4839s
    23    24 1.2515e+08    6  357 1.2658e+08 1.2515e+08  1.13%   465 4873s
    27    29 1.2515e+08    6  357 1.2658e+08 1.2515e+08  1.13%   424 4908s
    33    34 1.2515e+08    7  366 1.2658e+08 1.2515e+08  1.13%   387 5023s
    47    46 1.2515e+08  

In [ ]:
m

In [ ]:
print('Obj: %g' % m.objVal)

In [ ]:
for v in m.getVars():
    if(v.x != 0):
       print('%s   %g' % (v.varName, v.x))

In [ ]:
m = None

In [ ]:
disposeDefaultEnv()

# 